# Test of GraphSAGE
- use DGL
- predict `graphs`
- valid, test data are in the training dataset

In [30]:
import os
import dgl
import csv
import json
import torch
import torch as th
import pandas as pd
import torch.nn as nn
import dgl.nn as dglnn
import torch.nn.functional as F

from tqdm.notebook import tqdm
from sklearn.decomposition import PCA
from torch.optim import AdamW, lr_scheduler
from dgl.nn import GraphConv, GATConv, SAGEConv
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from transformers import get_linear_schedule_with_warmup

- check the GPU and assign the GPU by the best memory usage

In [3]:
import subprocess

def get_free_gpu():
    try:
        # Run nvidia-smi command to get GPU details
        _output_to_list = lambda x: x.decode('ascii').split('\n')[:-1]
        command = "nvidia-smi --query-gpu=memory.free --format=csv,nounits,noheader"
        memory_free_info = _output_to_list(subprocess.check_output(command.split())) 
        memory_free_values = [int(x) for i, x in enumerate(memory_free_info)]
        
        # Get the GPU with the maximum free memory
        best_gpu_id = memory_free_values.index(max(memory_free_values))
        return best_gpu_id
    except:
        # If any exception occurs, default to GPU 0 (this handles cases where nvidia-smi isn't installed)
        return 0

if torch.cuda.is_available():
    # Get the best GPU ID based on free memory and set it
    best_gpu_id = get_free_gpu()
    device = torch.device(f"cuda:{best_gpu_id}")
else:
    device = torch.device("cpu")
    print("there's no available GPU")

# device = torch.device(f"cuda:{1}")
print(device)


cuda:1


In [4]:
# device = torch.device(f"cuda:1")
# print(device)

## Fix the seed

In [5]:
import numpy as np
import torch
import random

#fix seed
def same_seeds(seed = 8787):
    torch.manual_seed(seed)
    # random.seed(seed) 
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

### Load the embedding

In [12]:
# Load embedding function
def load_embedding(input_embedding_name, model):
    if model.startswith('trans'):
        with open(input_embedding_name) as f:
            data = json.load(f)

        # trans family wouldn't consider the relation embedding -> directly use the word embedding
        # so the dimension of the node and the edge would be the same
        ent_embeddings = np.array(data['ent_embeddings.weight'])
        rel_embeddings = np.array(data['rel_embeddings.weight'])
        return ent_embeddings, rel_embeddings
    
    elif model == 'secureBERT':
        ent_embeddings = np.empty((0, 768), dtype=np.float32)
        for filename in sorted(os.listdir(input_embedding_name)):
            print(filename)

            if not filename.startswith('embeddings_chunk'):
                continue

            embedding = np.load(f'{input_embedding_name}/{filename}')

            print(ent_embeddings.shape, embedding.shape)

            ent_embeddings = np.concatenate((ent_embeddings, embedding), axis=0)
            print(filename, ent_embeddings.shape)

        print(f'Reducing entity embedding to ({DIM},)')
        print(ent_embeddings.shape, '->', end=' ')
        
        pca = PCA(n_components=DIM)
        ent_embeddings = pca.fit_transform(ent_embeddings)
        print(ent_embeddings.shape)

        # secureBERT would consider the edge embedding -> input is relation.npy
        # dimension of the node -> depends on us
        # dimension of the edge -> 26 (since PCA)
        rel_embeddings = np.load(f'{input_embedding_name}/relation.npy')
        print(f'Reducing relation embedding to ({len(rel_embeddings)},)')
        print(rel_embeddings.shape, '->', end=' ')
        pca = PCA(n_components=len(rel_embeddings))
        rel_embeddings = pca.fit_transform(rel_embeddings)
        print(rel_embeddings.shape)
        return ent_embeddings, rel_embeddings
    else:
        print('Error!!')
        return None
    

In [7]:
# import os
# import json
# import numpy as np
# from tqdm.notebook import tqdm
# from sklearn.decomposition import PCA

# # Load embedding function
# def load_embedding(input_embedding_name, model):
#     if model.startswith('trans'):
#         with open(input_embedding_name) as f:
#             data = json.load(f)
#         ent_embeddings = np.array(data['ent_embeddings.weight'])
#         rel_embeddings = np.array(data['rel_embeddings.weight'])
#         return ent_embeddings, rel_embeddings

#     elif model == 'secureBERT':
#         ent_embeddings = np.empty((0, 768), dtype=np.float32)
#         for filename in sorted(os.listdir(input_embedding_name)):
#             filepath = os.path.join(input_embedding_name, filename)
#             if not os.path.isfile(filepath) or not filename.startswith('embeddings_chunk'):
#                 continue

#             embedding = np.load(filepath)
#             print(filename)
#             print(ent_embeddings.shape, embedding.shape)
#             ent_embeddings = np.concatenate((ent_embeddings, embedding), axis=0)
#             print(filename, ent_embeddings.shape)
            
#             ent_embeddings = np.concatenate((ent_embeddings, embedding), axis=0)

#         # 对实体嵌入进行 PCA 降维
#         print(f'Reducing entity embedding to ({DIM},)')
#         pca_ent = PCA(n_components=DIM)
#         ent_embeddings = pca_ent.fit_transform(ent_embeddings)
#         print(f'Entity embeddings reduced: {ent_embeddings.shape}')
        
#         # 直接加载关系嵌入，不进行 PCA 降维
#         rel_embeddings = np.load(f'{input_embedding_name}/relation.npy')
#         print(f'Relation embeddings: {rel_embeddings.shape}')

#         return ent_embeddings, rel_embeddings

#     else:
#         print('Error!!')
#         return None


In [8]:
embedding_file = "../../data/4_embedding/synthesize/secureBERT"
input_filename = '../../data/source_data/before_embedding/3.10/all_graph_data.jsonl'

model = 'secureBERT'
DIM = 50

In [14]:
with open(input_filename, "r") as f:
    print("Loading the data...")

    # only process 40000 data from 400000 data
    wanted_data = 200000
    input_data = []
    for idx, line in tqdm(enumerate(f), total=wanted_data, desc="Loading"):
        if idx == wanted_data:
            break
        input_data.append(json.loads(line))
        
    print("FINISH...")

print(len(input_data))

Loading the data...


Loading:   0%|          | 0/200000 [00:00<?, ?it/s]

FINISH...
200000


In [15]:
model = embedding_file.split('/')[-1].split('_')[0]
print('Load Embedding ...')
ent_embeddings, rel_embeddings = load_embedding(embedding_file, model)

print('Process Embedding ...')
# if not tolist(), the original format is array -> error format
for data_point in tqdm(input_data, desc='Processing Embedding'):
    data_point['node_feat'] = [ent_embeddings[node_id].tolist() for node_id in data_point['node_feat']]
    data_point['edge_attr'] = [rel_embeddings[edge_id].tolist() for edge_id in data_point['edge_attr']]

Load Embedding ...
embeddings_chunk_0.npy
(0, 768) (160000, 768)
embeddings_chunk_0.npy (160000, 768)
embeddings_chunk_1.npy
(160000, 768) (160000, 768)
embeddings_chunk_1.npy (320000, 768)
embeddings_chunk_2.npy
(320000, 768) (160000, 768)
embeddings_chunk_2.npy (480000, 768)
embeddings_chunk_3.npy
(480000, 768) (20281, 768)
embeddings_chunk_3.npy (500281, 768)
relation.npy
Reducing entity embedding to (50,)
(500281, 768) -> (500281, 50)
Reducing relation embedding to (23,)
(23, 768) -> (23, 23)
Process Embedding ...


Processing Embedding:   0%|          | 0/200000 [00:00<?, ?it/s]

In [16]:
len(input_data[0]['node_feat'])

4

In [17]:
len(input_data[0]['edge_attr'])

7

In [18]:
input_data[0]['labels']

[0, 0, 0, 0, 0, 0, 0]

## Data Loader

In [19]:
class GraphDataset(Dataset):
    def __init__(self, data_list, device):
        self.data_list = data_list
        self.device = device

    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, idx):
        data = self.data_list[idx]
        return data

def collate(samples):
    data_list = samples
    batched_graphs = []
    for data in data_list:
        g = dgl.graph((th.tensor(data["edge_index"][0]), th.tensor(data["edge_index"][1])), num_nodes=data["num_nodes"])

        g.ndata['feat'] = th.tensor(data["node_feat"])
        g.edata['feat'] = th.tensor(data["edge_attr"])
        g.edata['label'] = th.tensor(data["labels"])  # Add edge labels to graph

        batched_graphs.append(g)
    
    return dgl.batch(batched_graphs)

In [20]:
# split 8:1:1 (train, valid, test)
train_data, remaining_data = train_test_split(input_data, test_size=0.2, random_state=42)
valid_data, test_data = train_test_split(remaining_data, test_size=0.5, random_state=42)


dataset_data = {
    'train': GraphDataset(train_data, device),
    'valid': GraphDataset(valid_data, device),
    'test': GraphDataset(test_data, device)
}

print("Datasets loaded and ready for training!")

Datasets loaded and ready for training!


- choose batch size

In [21]:
def create_dataloaders(batch_size, shuffle=True):
    dataloaders = {}
    for dataset_name, dataset in dataset_data.items():
        # do not shuffle the testing dataset
        if dataset_name == "test":
            dataloaders[dataset_name] = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=collate)    
        else:
            dataloaders[dataset_name] = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, collate_fn=collate)
    return dataloaders

dataloaders = create_dataloaders(200)

In [22]:
edge_embedding_dim = 0

# Assuming dataloaders is a dictionary with 'test' as one of the keys
for batch in dataloaders['train']:
    # Your batch processing code here
    print(batch, "\n")
#     print("edata:", batch.edata, '\n')
    print("edata['feat'] size:", batch.edata['feat'].shape, '\n')
    print("edata['label']:", batch.edata['label'])

    edge_embedding_dim = batch.edata['feat'].shape[1]

    break  # To break out after the first batch if needed

print("\n\nedge embedding dimension: ", edge_embedding_dim)

Graph(num_nodes=54917, num_edges=119235,
      ndata_schemes={'feat': Scheme(shape=(50,), dtype=torch.float32)}
      edata_schemes={'feat': Scheme(shape=(23,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64)}) 

edata['feat'] size: torch.Size([119235, 23]) 

edata['label']: tensor([0, 0, 0,  ..., 0, 0, 0])


edge embedding dimension:  23


- Turn the print message to a log file

In [23]:
import datetime

now = datetime.datetime.now()

formatted_time = now.strftime("%m%d_%H:%M")

log_file_path = f"./log_message/{formatted_time}_GraphSAGE_secureBERT_50-plusedge.log"

def add_log_msg(msg, log_file_path=log_file_path):
    with open(log_file_path, 'a') as f:
        f.write(f'{datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")}# {msg}\n')
    print(f'{datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")}# {msg}')

print(log_file_path)

./log_message/0124_01:22_GraphSAGE_secureBERT_50-plusedge.log


### Model

In [24]:
class GraphSAGE(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim):
        super(GraphSAGE, self).__init__()
        self.layer1 = dglnn.SAGEConv(in_dim, hidden_dim, 'pool')
        self.layer2 = dglnn.SAGEConv(hidden_dim, out_dim, 'pool')
        self.dropout = nn.Dropout(0.25)

    def forward(self, g, inputs):
        h = self.layer1(g, inputs)
        h = torch.relu(h)
#         h = self.dropout(h)
        h = self.layer2(g, h)
        return h

In [25]:
class MLPPredictor(nn.Module):
    def __init__(self, out_feats, out_classes, edge_embedding_dim):
        super().__init__()
        self.W = nn.Linear(out_feats*2 + edge_embedding_dim, out_classes)

    def apply_edges(self, edges, edge_feat):
#     def apply_edges(self, edges):

        h_u = edges.src['new_node_feat']
        h_v = edges.dst['new_node_feat']
        
        num_edges, edge_feat_dim = edge_feat.shape
#         print(num_edges, edge_feat_dim)
        
        h_e = edge_feat
        
        # concat 3 features
#         test = torch.cat([h_u, h_v, h_e],1)
#         print("with edge: ", test.shape)
        
#         test = torch.cat([h_u, h_v],1)
#         print("without edge: ", test.shape)
        
        score = self.W(torch.cat([h_u, h_v, h_e], 1))
#         score = self.W(torch.cat([h_u, h_v], 1))

        return {'score': score}


    def forward(self, graph, new_node_feat, edge_feat):
        with graph.local_scope():
            graph.ndata['new_node_feat'] = new_node_feat
#             graph.apply_edges(self.apply_edges)

            # 在 apply_edges 时传递 edge_feat
            graph.apply_edges(lambda edges: self.apply_edges(edges, edge_feat))
            return graph.edata['score']

In [26]:
class Model(nn.Module):
    def __init__(self, in_features, hidden_features, out_features, num_classes, edge_embedding_dim):
        super().__init__()
        self.sage = GraphSAGE(in_features, hidden_features, out_features)
        self.pred = MLPPredictor(out_features, num_classes, edge_embedding_dim)
      
    def forward(self, g, node_feat, edge_feat, return_logits=False):
        new_node_feat = self.sage(g, node_feat)
        logits = self.pred(g, new_node_feat, edge_feat)
        
        return logits

- Model Forward  

In [27]:
def model_fn(batched_g, model, criterion, device, count=1, which_type='train'):
    """Forward a batch through the model."""
#     batched_g, labels = data
    batched_g = batched_g.to(device)
    
    labels = batched_g.edata['label'].to(device)
    
#     logits = model(batched_g, batched_g.ndata['feat'].float())
    logits = model(batched_g, batched_g.ndata['feat'].float(), batched_g.edata['feat'].float())

    loss = criterion(logits, labels)

    output = torch.softmax(logits, dim=1)
    preds = output.argmax(1)
    
    # Compute accuracy
    accuracy = torch.mean((preds == labels).float())
        
    return loss, accuracy, preds

### Training

- Fix the seed and save the model.state_dict that contains the initial weight

In [ ]:
# seed = 8787
# same_seeds(seed)

# model = Model(in_features=50, hidden_features=64, out_features=128, num_classes=167)
# torch.save(model.state_dict(), 'model3_initial(graphsage)/initial_weight.pth')

In [ ]:
# # model.layer1.fc_self.weight
# model.sage.layer1.fc_self.weight

- Check if model really load the model_dict

In [ ]:
# model = Model(in_features=50, hidden_features=64, out_features=128, num_classes=167)
# model.load_state_dict(torch.load('model3_initial(graphsage)/initial_weight.pth'))
# model.sage.layer1.fc_self.weight

In [ ]:
# model.load_state_dict(torch.load('model3_initial(graphsage)/initial_weight.pth'))

### Training Loop

- define all the hyperparameters

In [31]:
seed = 8787
in_dim = DIM # dimension of the node feature
hidden_dim = 64
out_dim = 128
num_classes = 2 # for DARPA
edge_dim = edge_embedding_dim

lr = 5e-4

total_steps = 100
patience = 5
waiting = 0

In [32]:
model = Model(in_dim, hidden_dim, out_dim, num_classes, edge_dim)
best_model_path = "./checkpoint_graphSAGE/best_model_GraphSAGE_secureBERT_50-plusedge.pt"

optimizer = AdamW(model.parameters(), lr)

scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=36, eta_min=0, last_epoch=- 1, verbose=False)

criterion = nn.CrossEntropyLoss()
# criterion = torch.nn.BCEWithLogitsLoss()

In [33]:
same_seeds(seed)
model = model.to(device)
best_val_loss = float('inf')


# Training Part
for epoch in tqdm(range(total_steps)):
    # Train
    model.train()
    total_loss = 0.0
    total_accuracy = 0.0
    num_batches = 0
    
    for batched_g in tqdm(dataloaders['train'], desc="Training", position=0, leave=True):
        num_batches += 1
        loss, accuracy, _ = model_fn(batched_g, model, criterion, device, num_batches, which_type='train')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_accuracy += accuracy.item()

    scheduler.step()
    add_log_msg(f"total batches: {num_batches}")

    avg_loss = total_loss / num_batches
    avg_accuracy = total_accuracy / num_batches

    add_log_msg(f'Epoch {epoch} | Train Loss: {avg_loss:.4f} | Train Accuracy: {avg_accuracy:.4f}')

    
    # Validation Part
    model.eval()
    total_accuracy = 0.0
    total_loss = 0.0
    num_batches = 0


    with torch.no_grad():
        for batched_g in tqdm(dataloaders['valid'], desc="Validation", position=0, leave=True):
            loss, accuracy, _ = model_fn(batched_g, model, criterion, device, num_batches, which_type='validation')
            total_accuracy += accuracy.item()
            total_loss += loss.item()
            num_batches += 1

    avg_accuracy = total_accuracy / num_batches
    current_loss = total_loss / num_batches
    
    add_log_msg(f'Validation Loss: {current_loss:.4f} | Validation Accuracy: {avg_accuracy:.4f}\n')
    
            
    if current_loss < best_val_loss:
        best_val_loss = current_loss
        waiting = 0
        
        if os.path.exists(best_model_path):
            os.remove(best_model_path)
            add_log_msg("Find a better model!!")

        torch.save(model.state_dict(), best_model_path)
 
    else:
        waiting += 1
        if waiting >= patience:
            add_log_msg("============================== Early stopping ==================================")
            break

  0%|          | 0/100 [00:00<?, ?it/s]

Training:   0%|          | 0/800 [00:00<?, ?it/s]

01/24/2024, 01:28:47# total batches: 800
01/24/2024, 01:28:47# Epoch 0 | Train Loss: 0.1468 | Train Accuracy: 0.9644


Validation:   0%|          | 0/100 [00:00<?, ?it/s]

01/24/2024, 01:29:28# Validation Loss: 0.1003 | Validation Accuracy: 0.9679

01/24/2024, 01:29:28# Find a better model!!


...

01/24/2024, 04:16:07# Validation Loss: 0.0243 | Validation Accuracy: 0.9864



Training:   0%|          | 0/800 [00:00<?, ?it/s]

01/24/2024, 04:22:03# total batches: 800
01/24/2024, 04:22:03# Epoch 28 | Train Loss: 0.0239 | Train Accuracy: 0.9866


Validation:   0%|          | 0/100 [00:00<?, ?it/s]

01/24/2024, 04:22:45# Validation Loss: 0.0243 | Validation Accuracy: 0.9864

01/24/2024, 04:22:45# ============================== Early stopping ==================================


### test of valid and test part is ``graph``

- 60 APs in training x 10000times
- 5 APs in validation x 4 times
- 3 APs in test x 4 times
- Batch size = 4

In [34]:
# load the pretrained model
model.load_state_dict(torch.load(best_model_path))

model.to(device)
model.eval()

total = 0
correct = 0
count = 0

true_labels = []
predicted_labels = []

with torch.no_grad():
    for batched_g in tqdm(dataloaders['test'], desc="Testing", position=0, leave=True):

        loss, accuracy, predicted = model_fn(batched_g, model, criterion, device, count, which_type='test')
        labels = batched_g.edata['label'].to(device)
        
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        count += 1
        
add_log_msg(f'Test Accuracy: {100 * correct / total} %\n\n\n')

Testing:   0%|          | 0/100 [00:00<?, ?it/s]

01/24/2024, 13:20:15# Test Accuracy: 98.66362192795707 %





In [35]:
report_data = classification_report(true_labels, predicted_labels, output_dict=True)
report_df = pd.DataFrame(report_data).transpose()
report_df

,precision,recall,f1-score,support
0,0.990752,0.995460,0.993101,8.936242e+06
1,0.849996,0.734672,0.788137,3.129370e+05
accuracy,0.986636,0.986636,0.986636,9.866362e-01
macro avg,0.920374,0.865066,0.890619,9.249179e+06
weighted avg,0.985990,0.986636,0.986166,9.249179e+06


### If wanna output the excel file

In [ ]:
# report_data = classification_report(true_labels, predicted_labels, output_dict=True)
# report_df = pd.DataFrame(report_data).transpose()

# report_folder = 'classification_report'
# os.makedirs(report_folder, exist_ok=True)

# count = 0
# while True:
#     report_filename = f'classification_report-transE_50-graphSAGE-{count}.xlsx'
#     labels_filename = f'mapped_true_predicted_labels-transE_50-graphSAGE-{count}.xlsx'
    
#     report_path = os.path.join(report_folder, report_filename)
#     labels_path = os.path.join(report_folder, labels_filename)
    
#     if not os.path.exists(report_path) and not os.path.exists(labels_path):
#         break
#     count += 1

    
# report_df.to_excel(report_path, index_label='Label')

# labels_df = pd.DataFrame({'true_label': true_labels, 'predicted_label': predicted_labels})
# labels_df.to_excel(labels_path, index=False)

# add_log_msg(f"report path: {report_path}")
# add_log_msg(f"label path: {labels_path}")

# report = classification_report(true_labels, predicted_labels)
# add_log_msg(f"report:\n{report}")